# Amazon SageMaker Debugger - 組み込みルールの使用
[Amazon SageMaker](https://aws.amazon.com/sagemaker/) は、機械学習モデルを構築、トレーニング、ホストするためのマネージドプラットフォームです。 Amazon SageMaker Debugger は、モデルのにおける問題点をほぼリアルタイムで識別および検出することにより、トレーニング中に機械学習モデルをデバッグする機能を提供する新機能です。

このノートブックでは、TensorFlow トレーニングジョブ中に、SageMaker が提供する組み込みルールを使用する方法について説明します。

## Amazon SageMaker Debugger の機能

Amazon SageMaker Debugger を使用すると、トレーニング中に損失や精度などのスカラーを見るだけでなく、トレーニング中に「グラフを流れる」すべてのテンソルを完全に可視化できます。さらに、ルールを使用してほぼリアルタイムでトレーニングを監視し、トレーニングフローの不整合を検出するとアラートを提供します。

### 概念
* **Tensors**: 実行時の中間地点におけるトレーニングネットワークの状態
* **Debug Hook**： Hook は、Amazon SageMaker Debugger がトレーニングプロセスを調査し、必要なステップ間隔で要求されたテンソルをキャプチャするための構造
* **Rule**: Hook によってキャプチャされたテンソルを分析し、異常があれば報告するのに役立つ、Python コードとして実装された論理構造

これらの概念を念頭に置いて、Amazon SageMaker Debugger がデバッグを行う全体の流れを見ていきましょう。

### トレーニング中にテンソルを保存する

Debug Hook によってキャプチャされたテンソルは、ユーザーが指定した S3 の場所に保存されます。テンソルを保存するように Amazon SageMaker デバッガーを構成するには、2つの方法があります。

#### トレーニングスクリプトを変更しない
Amazon SageMaker が提供する[Deep Learning Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html) の 1.15 を使用している場合は、テンソルを保存するためにトレーニングスクリプトを変更する必要はありません。 Amazon SageMaker Debugger は、ジョブを作成するときに、Amazon SageMaker SDK の Tensorflow `Estimator` を介して提供する構成を使用して、指定した方法でテンソルを保存します。使用するスクリプトは、[src / mnist_zerocodechange.py](src/mnist_zerocodechange.py) で確認できます。これは、 `tf.estimator` インターフェースを使用するそのままのTensorFlowスクリプトであることに注意してください。 Amazon SageMaker デバッガは、 `tf.keras`、` tf.Estimator`、および `tf.MonitoredSession` インターフェイスのみをサポートすることに注意してください。サポートの詳しい説明は、[Amazon SageMaker Debugger with TensorFlow](https://github.com/awslabs/sagemaker-debugger/tree/master/docs/tensorflow.md) で入手できます。

#### テンソルを格納するためにスクリプトを調整する
他のコンテナの場合、トレーニングスクリプトに数行の変更を加える必要があります。 Amazon SageMakerデバッガーは、これらのテンソルをキャプチャして分析用に保存できるライブラリー `smdebug`を公開します。それは高度にカスタマイズ可能であり、異なる周波数で、おそらく他の構成で、必要な特定のテンソルを保存することができます。トレーニングスクリプトで選択したフレームワークでデバッガーライブラリを使用する方法の詳細については、[DeveloperGuide](https://github.com/awslabs/sagemaker-debugger/tree/master/docs) を参照してください。ここに、[src / mnist_byoc](src/mnist_byoc.py) に編成されたスクリプトの例があります。また、コンテナに `smdebug`ライブラリがインストールされていることを確認する必要があります。

### テンソルの分析

テンソルが保存されると、それらに対してデバッグ ***Rule*** を実行するように Amazon SageMaker デバッガーを構成できます。非常に広いレベルでは、ルールはトレーニング中に特定の条件を検出するために使用される Python コードです。アルゴリズムをトレーニングするデータサイエンティストが気にする可能性のある条件のいくつかは、勾配が大きくなりすぎたり、小さくなりすぎたり、過剰適合を検出したりすることを監視することです。 Amazon Sagemaker Debuggerには、特定のファーストパーティ（1P）ルールが事前にパッケージされています。ユーザーは、Amazon Sagemaker Debugger API を使用して独自のルールを作成できます。 Amazon Sagemaker デバッガーのAPIの完全なセットを使用して、たとえば SageMaker Notebook など、Rules コンストラクト の外で未加工のテンソルデータを分析することもできます。このノートブックは、トレーニングジョブで組み込みの SageMaker ルールを使用する方法を示し、インタラクティブな探索のためにこれらの API のピークを提供します。これらのAPIの詳細については、[開発者ガイド](https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md) を参照してください。

## セットアップ

この1回限りのセットアップに従って、ノートブックを起動し、Amazon SageMaker Debugger を使用できるようにします。本ノートブックで `smdebug` ライブラリを使用してインタラクティブな分析を実行していきます。

In [ ]:
! pip install smdebug

デバッガーを有効にしてSageMakerでTensorFlowモデルのトレーニングを開始しましょう。

## Amazon SageMaker Debugger を使用して SageMaker で TensorFlow モデルをトレーニングする

### フレームワークとしての Amazon SageMaker TensorFlow

Amazon SageMaker デバッガーを有効にして、このノートブックで TensorFlow モデルをトレーニングし、Amazon SageMaker Debugger ルールでトレーニングジョブをモニタリングします。 ここでは、フレームワークとして Amazon SageMaker [TensorFlow 1.15.0](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html) コンテナを使用します。

In [ ]:
import boto3
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

Amazon SageMaker Debugger のデモに必要なライブラリをインポートしましょう。

In [ ]:
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, rule_configs

次に、トレーニングを実行するための構成を定義します。 トレーニングの例として、MNIST データセットを使用した画像認識を使用します。

In [ ]:
# define the entrypoint script
entrypoint_script='src/mnist_zerocodechange.py'

hyperparameters = {
    "num_epochs": 3
}

### Estimator の設定

次に、TensorFlow Estimator を設定します。 Estimator に新しいパラメーターを追加して、Amazon SageMaker デバッガーを介してトレーニングジョブをデバッグできるようにしました。これらの新しいパラメーターについて、以下で説明します。

* **debugger_hook_config**：　この新しいパラメーターは、テンソルを書き込むローカルパスを受け入れるほか、テンソルをアップロードするS3 URIも受け入れます。 SageMaker は、実行中にこれらのテンソルを透過的にアップロードします。
* **Rule**：この新しいパラメーターは、このトレーニングジョブによって出力されたテンソルに対して評価するルールのリストを受け入れます。ルールの場合、Amazon SageMaker Debugger は2つのタイプをサポートします。
 * **SageMaker Rule**：　これらは、Amazon SageMaker のデータサイエンスおよびエンジニアリングチームによって特別にキュレーションされたルールであり、トレーニングジョブに対して評価することを選択できます。
 * **Custom Rule**：　オプションで、独自のルールを Python ソースファイルとして記述し、トレーニングジョブに対して評価することができます。このルールを評価するために Amazon SageMaker Debugger を提供するには、Rule source と Evaluator image の S3 の場所を提供する必要があります。
 
#### Amazon SageMaker ルールの使用
 
[この例](https://github.com/awslabs/sagemaker-debugger-rulesconfig)では、トレーニングに対して SageMaker ルールを使用して評価する方法を示します。 SageMaker ルールのリストとそれらを使用するのに最も適した構成を見つけることができます。

使用するルールは **VanishingGradient** と **LossNotDecreasing** です。名前が示すように、ルールは、トレーニング中にデバッグフックによってキャプチャされたテンソルに消失勾配があるかどうか、および損失が減少していないかどうかを評価しようとします。

In [ ]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()), 
    Rule.sagemaker(rule_configs.loss_not_decreasing())
]

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    base_job_name='smdebugger-demo-mnist-tensorflow',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    train_volume_size=400,
    entry_point=entrypoint_script,
    framework_version='1.15',
    py_version='py3',
    train_max_run=3600,
    script_mode=True,
    hyperparameters=hyperparameters,
    ## New parameter
    rules = rules
)

*Note that Amazon Sagemaker Debugger is only supported for py_version='py3' currently.*

Let's start the training by calling `fit()` on the TensorFlow estimator.

In [ ]:
estimator.fit(wait=True)

## 結果

`fit（）` を呼び出した結果、Amazon SageMaker デバッガーは2つのルール評価ジョブを開始し、トレーニングジョブと並行して、消失勾配と損失減少を監視しました。 ルール評価ステータスは、定期的にトレーニングログに表示されます。 ご覧のとおり、要約では、テンソルの消失勾配を報告するトレーニングにステップがありませんでした。 ただし、損失はステップ1900では減少していないことがわかりました。

In [ ]:
estimator.latest_training_job.rule_job_summary()

損失が減っていないかどうか、ルールジョブのログを見てみましょう。 そのために、このユーティリティ関数を使用して、ルールジョブログへのリンクを取得します。

In [ ]:
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
        """Helper function to get the rule job name with correct casing"""
        return "{}-{}-{}".format(
            training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
        )
    
def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(region, region, rule_job_name)


def get_rule_jobs_cw_urls(estimator):
    region = boto3.Session().region_name
    training_job = estimator.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]
    
    result={}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"])
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(rule_job_name, region)
    return result

get_rule_jobs_cw_urls(estimator)

## データ分析 - インタラクティブな探索
ジョブをトレーニングし、ルールによる自動分析を確認したので、Amazon SageMakerデバッガーの別の側面も見てみましょう。 リアルタイムで、または作業後に保存されたテンソルのインタラクティブな探索を実行できます。 ここでは、上記の仕事の事後分析に焦点を当てます。 単一のトレーニング実行を表すトライアルの概念を定義する `smdebug`ライブラリをインポートします。 上記のジョブのデバッガアーティファクトへのパスをフェッチする方法に注意してください。

In [ ]:
from smdebug.trials import create_trial
trial = create_trial(estimator.latest_job_debugger_artifacts_path())

何をプロットしたいかを知るために記録されたすべてのテンソルをリストできます。 これらの名前はそれぞれ、TensorFlowによって自動的に割り当てられるテンソルの名前です。 そのような名前が利用できないいくつかのフレームワークでは、レイヤーの名前と、それが重み、バイアス、グラデーション、入力または出力であるかどうかに基づいて名前を作成しようとします。

In [ ]:
trial.tensor_names()

また、 `smdebug`がトレーニングジョブから作成するデフォルトのコレクションによってテンソルを取得することもできます。 ここでは損失コレクションに関心があるので、次のように損失コレクションのテンソルの名前を取得できます。 Amazon SageMaker Debuggerは、重み、勾配、バイアス、損失などのデフォルトのコレクションを自動的に作成します。 テンソルからカスタムコレクションを作成することもできます。

In [ ]:
trial.tensor_names(collection="losses")

In [ ]:
import matplotlib.pyplot as plt
import re

# Define a function that, for the given tensor name, walks through all 
# the iterations for which we have data and fetches the value.
# Returns the set of steps and the values
def get_data(trial, tname):
    tensor = trial.tensor(tname)
    steps = tensor.steps()
    vals = [tensor.value(s) for s in steps]
    return steps, vals

def plot_tensors(trial, collection_name, ylabel=''):
    """
    Takes a `trial` and plots all tensors that match the given regex.
    """
    plt.figure(
        num=1, figsize=(8, 8), dpi=80,
        facecolor='w', edgecolor='k')

    tensors = trial.tensor_names(collection=collection_name)

    for tensor_name in sorted(tensors):
        steps, data = get_data(trial, tensor_name)
        plt.plot(steps, data, label=tensor_name)

    plt.legend(bbox_to_anchor=(1.04,1), loc='upper left')
    plt.xlabel('Iteration')
    plt.ylabel(ylabel)
    plt.show()
    
plot_tensors(trial, "losses", ylabel="Loss")